In [51]:
from abc import ABC, abstractmethod
from typing import TypeVar, Generic
from typing import Callable, List, Any
import re

In [52]:

Args = TypeVar('Args')
class Topic(Generic[Args], ABC):
    def __init__(self, retriver: Callable[[Args], str]):
        self.retriver: Callable[[Args], str] = retriver

    @abstractmethod
    def is_topic(self, text: str) -> bool:
        pass

    @abstractmethod
    def extract(self, start_text, print, input):
        pass

    @abstractmethod
    def get_name(self) -> str:
        pass

In [53]:
t = tuple[int,str]
class Weather(Topic[t]):
    def __init__(self, time_now: int, retriver: Callable[[t], str]):
        super().__init__(retriver)
        self.time_now = time_now

    def get_name(self) -> str:
        return "The weather"

    def is_topic(self, text: str) -> bool:
        return [w.lower() for w in text.split(" ")].__contains__("weather")
    def extract(self, start_text, print, input):
        
        
        time = None
        time_word = None
        city = None
        text = start_text
        
        while city == None:
            words = text.split(" ")
            lower_words = [w.lower() for w in text.split(" ")]
            new_city = self.find_city(words, lower_words)
            if new_city != None:
                city = new_city
            new_time, new_time_word = self.find_time(words, lower_words, text)
            if new_time != None:
                time = new_time
                time_word = new_time_word

            if time == None and city == None:
                text = input("Sorry, I could not unstand you, where and when do you want to know the weather?")
            elif(city == None):
                text = input(f"You want to know the weather {time_word}, but I need your location as well")

        if(time == None):
            current_weather = self.retriver((0,city))
            if current_weather != None:
                print(f"The weather in {city} right now is {current_weather}")
            else:
                print(f"Could not find current weather for {city}")
        else:
            weather = self.retriver((time,city))
            if weather != None:
                print(f"The weather in {city} {time_word} is {weather}")
            else:
                print(f"Could not find current weather for {city} {time_word}")
       
    
    def find_city(self, words, lower_words):
        city = None
        in_indecies = [i for i, w in enumerate(lower_words) if w == "in"]
        if len(in_indecies) != 0:
            if in_indecies[0] != len(words) and words[in_indecies[0]+1][0].isupper():
                city =  words[in_indecies[0]+1]
        return city
    def find_time(self, words, lower_words, text):
        time = None
        time_word = None
        in_indecies = [i for i, w in enumerate(lower_words) if w == "in"]
        if lower_words.__contains__("tomorrow"):
            time_word = "tomorrow"
            time = self.time_now + 24
        elif  lower_words.__contains__("now"):
            time_word = "now"
            time = self.time_now
        elif lower_words.__contains__("today"):
            time_word = "today"
            time = self.time_now
        elif text.__contains__(":"):
            time_word = [w for w in words if w.__contains__(":")][0]
            hour = int(time_word.split(":")[0])
            time = hour - self.time_now
        elif len(in_indecies) > 1:
            if len(words) > in_indecies[1] + 2 and  words[in_indecies[1] + 1].isnumeric() and  words[in_indecies[1] + 2][:3] == "day":
                time_word = " ".join(words[in_indecies[1]:in_indecies[1]+3])
                time = self.time_now + int(words[in_indecies[1] + 1]) * 24
        return time, time_word


In [54]:
weathers = {(0, "A") : "Rain",(20, "B"): "Sun", (48, "B"): "Cloud", (24, "B"): "Hale" }
weatehr_getter = lambda args: weathers.get(args,None)
weatehr_getter((0, "A"))

'Rain'

In [55]:
a = Weather(0, weatehr_getter)
a.is_topic("What is the weather in A now")

True

In [56]:
a.extract("What is the weather in B in 1 day", print, input)

The weather in B in 1 day is Hale


In [57]:
import re

class Restaurant(Topic):
    def __init__(self, retriver: Callable[[Args], str]):
        super().__init__(retriver)

    def is_topic(self, text: str) -> bool:
        word_list = ["food", "restaurant", "eat"]

        text = [w.lower() for w in text.split()]

        for word in text:
            if word in word_list:
                return True

        return False
    
    def find_location(self, text: str):
        word_list = ["in"]
        pattern = r'[^a-zA-Z0-9\s]'  
        text = re.sub(pattern, '', text)
        text = [w.lower() for w in text.split()]

        for i, word in enumerate(text):
            if word in word_list:
                if i >= len(text)-1:
                    return None
                return text[i+1]
        return None

    def find_food_type(self, text: str):
        word_list = ["food", "restaurant"]
        pattern = r'[^a-zA-Z0-9\s]'  
        text = re.sub(pattern, '', text)
        text = [w.lower() for w in text.split()]

        for i, word in enumerate(text):
            if word in word_list:
                if i == 0:
                    return None
                return text[i-1]
        return None

    def extract(self, start_text, print, input):
        text = start_text
        food_type = None
        location = None

        while food_type == None or location == None:

            if food_type == None:
                food_type = self.find_food_type(text)
            if location == None:
                location = self.find_location(text)

            if food_type == None and location == None:
                text = input("I don't understand, what type of food are you looking for and where?")
            elif food_type == None:
                text = input(f"I don't understand, what type of food are you looking for in {location}?")
            elif location == None:
                text = input(f"I don't understand, you are looking for a {food_type} restaurant in what location?")

        street = self.retriver((food_type, location))        

        if street != None:
            print(f"The closest {food_type} restaurant in {location} is at {street}")
        else:
            print(f"Could not find any {food_type} restaurants in {location}.")

    def get_name(self):
        return "Finding restaurants"
             
        


In [58]:

class Bus(Topic):
    def __init__(self, retriver: Callable[[Args], str]):
        super().__init__(retriver)

    def is_topic(self, text: str) -> bool:
        word_list = ["bus", "tram"]

        text = [w.lower() for w in text.split()]

        for word in text:
            if word in word_list:
                return True

        return False

    def find_stop(self, text: str):
        before_list = ["from", "leave"]
        after_list = ["stop"]

        pattern = r'[^a-zA-Z0-9\s]'  
        text = re.sub(pattern, '', text)

        text = [w.lower() for w in text.split()]
        

        for i, word in enumerate(text):
            if word in before_list:
                if i == len(text)-1:
                    return None
                return text[i+1]
            
            if word in after_list:
                if i == 0:
                    return None
                return text[i-1]
        return None

    def extract(self, start_text, print, input):
        text = start_text
        stop = None

        while stop == None:

            stop = self.find_stop(text)

            if stop == None:
                text = input("I don't understand, what stop?")
        
        vehicle, time = self.retriver(stop)

        print(f"The next {vehicle} leaving {stop} departs at {time}.")

    def get_name(self):
        return "Bus/Tram"


In [59]:
class Dialog_bot:
    def __init__(self, topics: List[Topic[Any]]) -> None:
        self.topics = topics
    def start(self):
        start_text = input("Hello, how could I hellp you?")
        while True:
            current_topic = None
            while current_topic == None:
                hits = [t for t in self.topics if t.is_topic(start_text)]
                if len(hits) == 0:
                    start_text = input("Sorry, I do not undersatnd, could you rephrase")
                elif len(hits) > 1:
                    topic_names = [t.get_name() for t in self.topics]
                    s = f"{', '.join(topic_names[:-1]) } and {topic_names[-1]}"
                    start_text = input("It seems like you are talking about {s}")
                else:
                    current_topic = hits[0]
            current_topic.extract(start_text, print, input)
            start_text = input("IS there anything else I help you with?")
        

In [60]:
restaurants = {("burger", "gothenburg") : "Something1", ("thai", "gothenburg"): "Something2" }
restaurant_getter = lambda args: restaurants.get(args, None)
test = Restaurant(restaurant_getter)

In [61]:
stops = {"chalmers" : ("tram", "13:14")}
stop_getter = lambda args: stops.get(args,"Could not find :(")
bus = Bus(stop_getter)


In [63]:
topics = [Weather(0, weatehr_getter), Restaurant(restaurant_getter), Bus(stop_getter)]
bot = Dialog_bot(topics)
bot.start()

The weather in A right now is Rain


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fff6b3e9-ae6f-4702-985b-52f8a0352010' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>